In [ ]:
pip install warcio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 5.8 MB/s eta 0:00:00


In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 13.4 MB/s eta 0:00:00


In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=f7e0ec56c0e10d75ed1e9c989fe6f5735df980e3e43f355493d48498a7791231
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
import os
import warcio
import re
import nltk
import gzip
import boto3
nltk.download('words')
from warcio.archiveiterator import ArchiveIterator
from nltk.corpus import words
spark = SparkSession.builder \
    .appName("myApp") \
    .config("spark.driver.memory", "64g") \
    .config("spark.executor.memory", "64g") \
    .getOrCreate()
spark

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
schema = StructType([
    StructField("WARC-Type", StringType(), True),
    StructField("WARC-Target-URI", StringType(), True),
    StructField("WARC-Date", StringType(), True),
    StructField("WARC-Record-ID", StringType(), True),
    StructField("WARC-Refers-To", StringType(), True),
    StructField("WARC-Block-Digest", StringType(), True),
    StructField("WARC-Identified-Content-Language", StringType(), True),
    StructField("Content-Type", StringType(), True),
    StructField("Content-Length", IntegerType(), True),
    StructField("Content", StringType(), True)
])

In [ ]:
ACCESS_KEY=""
SECRET_KEY=""

In [ ]:
s3 = boto3.resource('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
for bucket in s3.buckets.all():
    print(bucket.name)

apbd5254
aws-emr-resources-411926377614-us-east-1
aws-logs-411926377614-us-east-1
elasticbeanstalk-us-east-1-411926377614
elasticbeanstalk-us-west-2-411926377614


In [ ]:
keys=[]
for obj in s3.Bucket('apbd5254').objects.all():
    keys.append(obj.key)

In [ ]:
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
obj = s3.get_object(Bucket= 'apbd5254', Key= 'keys_novdec_22.txt')

In [ ]:
file = obj['Body']

In [ ]:
file = file.read()

In [ ]:
file = file.decode("utf-8")

In [ ]:
file = file.splitlines()

In [ ]:
for i in range(200): #1TB
    records = []
    obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i])
    with gzip.open(obj['Body'], 'r') as stream:
        for record in ArchiveIterator(stream):
            _record = dict(record.rec_headers.headers)
            _record['Content'] = record.raw_stream.read().decode('utf-8')
            records.append(_record)
    spark_df = spark.createDataFrame(records)
    tokenizer = RegexTokenizer(inputCol="Content", outputCol="words", pattern="\\W")
    tokenized_df = tokenizer.transform(spark_df)
    stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    filtered_df = stopwords_remover.transform(tokenized_df)
    english_words = set(words.words())
    def filter_english_words(words):
        return [word for word in words if word.lower() in english_words]
    filter_english_words_udf = udf(filter_english_words, ArrayType(StringType()))
    filtered_english_words_df = filtered_df.withColumn("english_words", filter_english_words_udf("filtered"))
    english_words = set(words.words())
    def filter_english_words(words):
        return [word for word in words if word.lower() in english_words]
    filter_english_words_udf = udf(filter_english_words, ArrayType(StringType()))
    filtered_english_words_df = filtered_df.withColumn("english_words", filter_english_words_udf("filtered"))
    vectorizer = CountVectorizer(inputCol="english_words", outputCol="rawFeatures", vocabSize=5000, minDF=5.0)
    vectorized_df = vectorizer.fit(filtered_english_words_df).transform(filtered_english_words_df)
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idf_model = idf.fit(vectorized_df)
    tfidf_vectors = idf_model.transform(vectorized_df)
    num_topics = 5
    lda = LDA(k=num_topics, maxIter=10)
    lda_model = lda.fit(tfidf_vectors)
    vectorizer_model = vectorizer.fit(filtered_english_words_df)
    vocab = vectorizer_model.vocabulary
    topics = lda_model.describeTopics(maxTermsPerTopic=20)
    topic_terms_udf = udf(lambda indices: [vocab[i] for i in indices], ArrayType(StringType()))
    topics_with_terms = topics.withColumn("topic_terms", topic_terms_udf("termIndices"))
    n = 3
    top_topics = topics_with_terms.select("topic", "topic_terms").limit(n).collect()
    top_topics_list = [(row.topic, row.topic_terms) for row in top_topics]
    string = "The top " + str(n) + " topics are: " + str(top_topics_list)
    s3.put_object(
    Body=string, 
    Bucket='apbd5254', 
    Key='TopicModel/'+str(i) +'.txt'
  )